<a href="https://colab.research.google.com/github/DavoodSZ1993/RL/blob/main/08_DQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import gym
import math                                  # Provides access to the mathematical functions
import random                                # Implements pseudo-random number generators for various distributions
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count                 # Make an iterator that returns evenly spaced values starting with number start
from PIL import Image                       # Provides a number of factory functions including functions to load images from files

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

## `collections.namedtuple()` Factory Function for Tuples with Named Fields.
Named tuples assign meaning to each position in a tuple and allow for more readable, self-documenting code. 

```
>>> # Basic example
>>> Point = namedtuple('Point', ['x', 'y'])
>>> p = Point(11, y=22)     # instantiate with positional or keyword arguments
>>> p[0] + p[1]             # indexable like the plain tuple (11, 22)
33
>>> x, y = p                # unpack like a regular tuple
>>> x, y
(11, 22)
>>> p.x + p.y               # fields also accessible by name
33
>>> p                       # readable __repr__ with a name=value style
Point(x=11, y=22)
```

## `collections.deque()`:
Returns a new deque object inialized left-to-right (append()) with data from iterable. if iterable is not specified, the new deque is empty. 

```
>>> from collections import deque
>>> d = deque('ghi')                 # make a new deque with three items
>>> for elem in d:                   # iterate over the deque's elements
...     print(elem.upper())
G
H
I
```